In [ ]:
import numpy as np
import pymongo as pm

### connect to db

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['3dObjects']
coll = db['chairs_chatbox']

### basic details about dataset


In [ ]:
###### how many total after we started the chairs1k production sample
wave1 = coll.find({'$and': [{'iterationName':'close_only'},{'eventType':'clickedObj'},{'time':{'$gt':1507674252874}}]}).sort('intendedName')
wave2 = coll.find({'$and': [{'iterationName':'chairs1k'},{'eventType':'clickedObj'},{'time':{'$gt':1507674252874}}]}).sort('intendedName')
print str(wave1.count() + wave2.count()) + ' chair trials in dataset.'

### loop through all games and build group data csv

In [ ]:
## TODO: encoding for worker ID so that we can always map from that anonymized ID to the actual AMT worker ID

In [ ]:
## get for iterationName = 'chairs1k'

########## get list of games
a = coll.find({'$and': [{'iterationName':'chairs1k'},
                    {'eventType':'chatMessage'},
                    {'time':{'$gt':1507674252874}}
                   ]}).sort('time')
games = a.distinct('gameid')

chairA = []
chairB = []
chairC = []
intendedName = []
clickedObj = []
chatMessage = []
message_counter = []
gameid = []
click_time = []
trialNum = []
condition = []
correct = []
chat_time = []
chat_RT = []
role = []

for i,game in enumerate(games):    
    print str(i) + ' ' + game
    clicks = coll.find({'$and': [{'iterationName':'chairs1k'},
                        {'eventType':'clickedObj'},
                        {'time':{'$gt':1507674252874}},
                        {'gameid': game}
                       ]}).sort('trialNum')  
    
    for click in clicks:
        
        currTrial = click['trialNum']        
        
        chats = coll.find({'$and': [{'iterationName':'chairs1k'},
                            {'eventType':'chatMessage'},
                            {'time':{'$gt':1507674252874}},
                            {'gameid': game},
                            {'trialNum': currTrial}
                           ]}).sort('time')        
        
        for chat in chats:
            chat_time.append(chat['time'])
            chat_RT.append(chat['reactionTime'])
            chatMessage.append(chat['text'])
            role.append(chat['role'])
            intendedName.append(chat['intendedName'])
            
            clickedObj.append(click['clickedName'])
            trialNum.append(click['trialNum'])        
            chairA.append(click['object1Name'])
            chairB.append(click['object2Name'])
            chairC.append(click['object3Name'])  
            click_time.append(click['time'])
            gameid.append(click['gameid'])
            condition.append(click['condition'])
            correct.append(click['correct'])        

In [ ]:
########## get list of close-only-games
a = coll.find({'$and': [{'iterationName':'close_only'},
                    {'eventType':'chatMessage'},
                    {'time':{'$gt':1507674252874}}
                   ]}).sort('time')
games = a.distinct('gameid')

for i,game in enumerate(games):    
    print str(i) + ' ' + game
    clicks = coll.find({'$and': [{'iterationName':'close_only'},
                        {'eventType':'clickedObj'},
                        {'time':{'$gt':1507674252874}},
                        {'gameid': game}
                       ]}).sort('trialNum')  
    
    for click in clicks:
        
        currTrial = click['trialNum']        
        
        chats = coll.find({'$and': [{'iterationName':'close_only'},
                            {'eventType':'chatMessage'},
                            {'time':{'$gt':1507674252874}},
                            {'gameid': game},
                            {'trialNum': currTrial}
                           ]}).sort('time')        
        
        for chat in chats:
            chat_time.append(chat['time'])
            chat_RT.append(chat['reactionTime'])
            chatMessage.append(chat['text'])
            role.append(chat['role'])
            intendedName.append(chat['intendedName'])
            
            clickedObj.append(click['clickedName'])
            trialNum.append(click['trialNum'])        
            chairA.append(click['object1Name'])
            chairB.append(click['object2Name'])
            chairC.append(click['object3Name'])  
            click_time.append(click['time'])
            gameid.append(click['gameid'])
            condition.append(click['condition'])
            correct.append(click['correct']) 

In [ ]:
import pandas as pd

In [ ]:
X = pd.DataFrame([chairA,
                chairB,
                chairC,
                intendedName,
                clickedObj,
                chatMessage,
                gameid,
                click_time,
                trialNum,
                condition,
                correct,
                chat_time,
                chat_RT,
                role])

X = X.transpose()
X.columns = ['chair_a','chair_b','chair_c','target_chair','selected_chair','text','gameid',
            'click_time','trial_num','context_condition','correct','chat_time','chat_RT', 'communication_role']

In [ ]:
X.to_csv('chairs1k_group_data.csv',encoding='utf-8')

In [ ]:
X.shape

### Sanity checks on data organization

In [ ]:
## the three chairs ID's should be unique
from collections import defaultdict
all_triplets = defaultdict(list)
for i,d in X.iterrows():

    tokens_1 = d['chair_a'].split('_')
    tokens_2 = d['chair_b'].split('_')
    tokens_3 = d['chair_c'].split('_')
    
    triplets = []        
    triplets.append(tokens_1[-1])    
    triplets.append(tokens_2[-1])
    triplets.append(tokens_3[-1])
    assert(len(np.unique(triplets)) == 3)
    
    assert(d['context_condition'] == tokens_1[0] == tokens_2[0] == tokens_3[0])
    
    all_triplets[(d['context_condition'], d['target_chair'])].append(np.array(triplets, dtype=object))
    
#     assert d['clickedObj'] in  [d['chairsA'], d['chairsB'], d['chairsC']]

#     assert(d['target_chair'] in  [d['chairsA'], d['chairsB'], d['chairsC']])
    
#     if d['intendedName'] == d['clickedObj']:
#         assert(d['correct'])
#     else:
#         assert(not d['correct'])
    
    ## check the total number of chair ID's

    ## check the total number of unique triplets (where target identity matters AND where it does not matter)

    


In [ ]:
c_close = 0 
c_far = 0
close_triplets = []
far_triplets = []
for l in all_triplets.keys():
    
#     a  = all_triplets[l][0].split('_')[-1]
#     b = all_triplets[l][1].split('_')[-1]
#     c = all_triplets[l][2].split('_')[-1]
    
    if l[0] == 'close':
        c_close +=1
        close_triplets.append(np.sort(all_triplets[l]))
    elif l[0] == 'far':
        c_far += 1
        far_triplets.append(np.sort(all_triplets[l]))
    else:
        assert(False)
        

In [ ]:
far_triplets = np.vstack(close_triplets)

In [ ]:
len(np.unique(far_triplets))